In [16]:
from bluepy.btle import Scanner, DefaultDelegate, Peripheral, UUID
import struct
import itertools
import pandas as pd
import time
import matplotlib.pyplot as plt

In [17]:
acc_arr = []

In [23]:
global ACC_ARR

class ScanDelegate(DefaultDelegate):
    def __init__(self):
        DefaultDelegate.__init__(self)

    def handleNotification(self, cHandle, data):
        #print(cHandle)
        print(struct.unpack('hhhhhhhh', data))
        #if cHandle == 13:
        #    ACC_ARR.append(struct.unpack('fffff', data))
        #if cHandle == 12:
        #    ACC_ARR.append(struct.unpack('hhhhhhhh', data))
        #if cHandle == 15:
        #    gyro_arr.append(struct.unpack('ffff', data))
        #if len(acc_arr)>100:
        #    print('writing file')
            #pd.DataFrame(acc_arr).to_csv('/home/asya/Desktop/Gyro_Data/Data_100hz/fle_{}.csv'.format(str(pd.Timestamp.now())))
        #    f = f+1
        #    ACC_ARR = []
        

class GAmonitor(object):

    SERVICE_UUIDS = [
        UUID('c0be9c58-a717-4e18-828c-a41836f0c7e5'), # Sensors
    ]
    
    CHARACTERISTIC_UUIDS = {
    #    '9a067338-1da7-4e0f-8b4a-1e445e1e2df8': 'ACC',
        '9a067338-1da7-4e0f-8b4a-1e445e1e2df9': 'ACC'
    }

    NOTIFICATION_CHARACTERISTIC_UUIDS = [
        'ACC',
        #'GYRO',
        ]

    # Notification data
    NOTIFICATION_ON = struct.pack("BB", 0x01, 0x00)
    NOTIFICATION_OFF = struct.pack("BB", 0x00, 0x00)
    ACC_ARR = []
    
    def __init__(self, mac_address):
        self.macAddress = mac_address
        self.delegate=ScanDelegate()

    def set_delegate(self, delegate):
        self.delegate = delegate

    def connect(self):
        print('getting peripheral')
        self.peripheral = Peripheral(self.macAddress, addrType='public')
        # Retrieve all characteristics from desired services and map them from their UUID
        self.peripheral.getServices()
        svc = self.peripheral.getServiceByUUID("c0be9c58-a717-4e18-828c-a41836f0c7e5")
        characteristics = {svcid: svc.getCharacteristics()[i] for svcid, i in zip(self.CHARACTERISTIC_UUIDS, range(2))}
        
        print(characteristics)
        #characteristics = list(itertools.chain(*[self.peripheral.getServiceByUUID(uuid).getCharacteristics() for uuid in GAmonitor.SERVICE_UUIDS]))
        
        #characteristics = [svc.getCharacteristics()[0] for svcid in self.CHARACTERISTIC_UUIDS]
        #characteristics = dict((c.uuid, c) for c in characteristics)
        # Store each characteristic's value handle for each characteristic name
        self.characteristicValueHandles = dict((name, characteristics[uuid].getHandle()) for uuid, name in self.CHARACTERISTIC_UUIDS.items())
        
        # Subscribe for notifications
        for name in self.NOTIFICATION_CHARACTERISTIC_UUIDS:
            print('Enabling notification: ')
            self.peripheral.writeCharacteristic(self.characteristicValueHandles[name]+1, self.NOTIFICATION_ON, True)
            print(name)
            print(self.characteristicValueHandles[name]+1)
            
        self.peripheral.setDelegate(self.delegate)

    def wait_for_notifications(self, timeout):
        self.peripheral.waitForNotifications(timeout)

In [24]:
GA = GAmonitor(mac_address = "f6:29:60:b4:99:4b") #"f6:29:60:b4:99:4b")
GA.connect()

getting peripheral
{'9a067338-1da7-4e0f-8b4a-1e445e1e2df9': <bluepy.btle.Characteristic object at 0x7f499d9bff50>}
Enabling notification: 
ACC
13


In [33]:
GA.wait_for_notifications(1.0)

(2813, 3, 35, -2, 12, -758, 383, 476)


In [35]:
GA.peripheral.disconnect()

In [5]:
#global acc_arr
def get_data(acc_arr):
    f = 0
    print('Reading data')
    print(len(acc_arr))
    pd.DataFrame(acc_arr).to_csv('/home/asya/Desktop/Gyro_Data/Data_100hz/fle_{}_{}.csv'.format(f, str(pd.Timestamp.now())))
    #acc_arr = []
    while(1):
        GA.wait_for_notifications(1.0)
        #print(acc_arr)
        print(len(acc_arr))
        if len(acc_arr)>100:
            print('writing file')
            pd.DataFrame(acc_arr).to_csv('/home/asya/Desktop/Gyro_Data/Data_100hz/fle_{}_{}.csv'.format(f, str(pd.Timestamp.now())))
            f = f+1
            acc_arr = []

In [6]:
len(acc_arr)

0

In [ ]:
ls /home/asya/Desktop/Gyro_Data/Data_100hz/

In [10]:
if 1:
    while(1):
        try: 
            GA = GAmonitor(mac_address = "f6:29:60:b4:99:4b") #"f6:29:60:b4:99:4b")
            GA.connect()
            while(1):
                GA.wait_for_notifications(1.0)
            #get_data(acc_arr)
            #'looping'
        except Exception as e:
            print(e)
            print('error')

getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
{'9a067338-1da7-4e0f-8b4a-1e445e1e2df9': <bluepy.btle.Characteristic object at 0x7f72709ad910>}
Enabling notification: 
ACC
13
local variable 'acc_arr' referenced before assignment
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to peripheral f6:29:60:b4:99:4b, addr type: public
error
getting peripheral
Failed to connect to pe

KeyboardInterrupt: 

In [ ]:
acc_arr = []
t_arr_1 = []
for _ in range(100):
    tic = time.time()
    GA.wait_for_notifications(1.0)
    t_arr_1.append(1000*(time.time()-tic))

In [ ]:
sum(t_arr_1)

In [ ]:
if 0:
    df_gyro = pd.DataFrame(gyro_arr)
    df_gyro['t'] = round(df_gyro[3])
    df_gyro['t'] = df_gyro['t'] - df_gyro['t'].loc[0]

In [ ]:
#acc_arr

In [ ]:
df_acc = pd.DataFrame(acc_arr)
df_acc['t'] = round(df_acc[0])
df_acc['t'] = df_acc['t'] - df_acc['t'].loc[0]
df_acc['t_diff'] = df_acc['t'].diff()

In [ ]:
df_acc['t_diff'].describe()

##### df_acc['t_rec'] = t_arr_1
df_acc['t_rec'] = 1000*df_acc['t_rec']

In [ ]:
df_acc[1].plot()
(100*df_acc[2]).plot()
(100*df_acc[3]).plot()
(100*df_acc[4]).plot()

In [ ]:
GA.peripheral._helper.stdout.readline()

In [ ]:
t_arr = []
for x in range(1000):
    tic = time.time()
    #GA.peripheral._waitResp('ntfy')
    val = GA.peripheral._helper.stdout.readline()
    #print(1000*(time.time()-tic))
    #if 1000*(time.time()-tic) > 1:
    #    print(x)
    toc = time.time()
    t_arr.append([val,1000*(toc-tic)])
    #time.sleep(max(0, 0.008 - (toc-tic)))

In [ ]:
pd.DataFrame(t_arr)[1].plot()

In [ ]:
#(1/pd.DataFrame(t_arr)[1]).plot()

In [ ]:
print(1/(pd.DataFrame(t_arr)[1].mean()))
print((pd.DataFrame(t_arr)[1].mean()))

In [ ]:
df_test = pd.DataFrame(t_arr)

In [ ]:
#df_test[0].values

In [ ]:
df_test.loc[df_test[1]>0.05]

In [ ]:
df_test[1].describe()

In [ ]:
import numpy as np